# Table Header Detection

## Requirements
- Conda
- MongoDB instance
- PyMongo (will be installed by the notebook)

In [1]:
#import sys
# !conda install --yes --prefix {sys.prefix} pymongo
# !conda install --yes --prefix {sys.prefix} premailer
# pip install premailer
#!{sys.executable} -m pip uninstall numpy
#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install pandas

import os
import json
import re
#import pandas as pd
from pymongo import MongoClient
import requests
from bs4 import BeautifulSoup
from bs4.element import Tag
#from cssutils import parseStyle
#from premailer import Premailer
import time
from dateutil.parser import parse
import math


something


## Loading the seed data into mongoDB
- initial dataset [Wikipedia TabEL dataset](http://websail-fe.cs.northwestern.edu/TabEL/)
- dataset is lacking of some styling information
- we're crawling the wikipedia pages on our own
  - that should be feasible since we have to use labeled data only (both for training & testing)
  - we're taking the TabEL dataset pageID's as starting point, since we know that there should be at least one relational table

Each line of the TabEL dataset contains one JSON object representing a single table. However, the JSON objects are not contained within a JSON array. We need to wrap the single tables into an array first before we can parse the file as a whole.

In [4]:
def wrapJSONObjectLineIntoTable(inputFilePath, outputFilePath):
    inputFile = open(inputFilePath, 'r')
    outputFile = open(outputFilePath, 'w')

    outputFile.write('[')

    previousLine = False
    for tableLineJsonObject in inputFile:
        if (previousLine):
            outputFile.write(previousLine + ',')
        previousLine = tableLineJsonObject
    if (previousLine):
        outputFile.write(previousLine)

    outputFile.write(']')

    inputFile.close()
    outputFile.close()

Check if TabEL dataset has been transformed into an array before. If not, we want to parse it now.

In [12]:
inputFilePath = os.path.join('data', 'wikipedia_0_5000.json')
outputFilePath = os.path.join('data', 'wikipedia_0_5000_fixed.json')
if not os.path.isfile(outputFilePath):
    wrapJSONObjectLineIntoTable(inputFilePath, outputFilePath)

Parse JSON Array

In [83]:
tabEL = pd.read_json(os.path.join('data', 'wikipedia_0_5000_fixed.json'))
tabEL.head()

,_id,numCols,numDataRows,numHeaderRows,numericColumns,order,pgId,pgTitle,sectionTitle,tableCaption,tableData,tableHeaders,tableId
0,10000032-1,4,11,1,[1],0.535975,10000032,Mid Antrim (Northern Ireland Parliament consti...,Members of Parliament,Members of Parliament,"[[{'cellID': -1, 'textTokens': [], 'text': '',...","[[{'cellID': -1, 'textTokens': [], 'text': 'El...",1
1,1000006-1,4,21,1,[],0.856769,1000006,Römer (crater),Satellite craters,Satellite craters,"[[{'cellID': -1, 'textTokens': [], 'text': 'A'...","[[{'cellID': -1, 'textTokens': [], 'text': 'Rö...",1
2,10000088-1,2,1,3,[],0.318258,10000088,Whispermoon,,Track listing,"[[{'cellID': -1, 'textTokens': [], 'text': 'Al...","[[{'cellID': -1, 'textTokens': [], 'text': 'Pr...",1
3,10000218-1,2,6,1,[],0.553872,10000218,Khalsa Diwan Society Vancouver,First executive committee,First executive committee,"[[{'cellID': -1, 'textTokens': [], 'text': 'Pr...","[[{'cellID': -1, 'textTokens': [], 'text': 'Ti...",1
4,10000228-1,2,7,1,[1],0.951118,10000228,Julien Leparoux,Year-end charts,Year-end charts,"[[{'cellID': -1, 'textTokens': [], 'text': 'Na...","[[{'cellID': -1, 'textTokens': [], 'text': 'Ch...",1


Get 1000 unique page IDs and fetch the HTML content for it.

In [101]:
pageIDSample = tabEL[['pgId']].sample(n=1000)

## Crawl the wikipedia pages and fetch all occurring tables
We use the pageID's from the TabEL dataset and crawl the wikipedia html. One page might include multiple tables. We only extract HTML tables with the class `wikitable`. The style from the CSS file gets parsed into inline style.

In [8]:
BASE_URL = 'https://en.wikipedia.org/'
wikipediaCSSFilePath = os.path.join('data', 'wikipedia.css')
instance = Premailer(base_url=BASE_URL)

cssFilePath = os.path.join('data', 'wikipedia.css')
cssFile = open(cssFilePath, 'r')
css = cssFile.read()
style = '<style>' + css + '</style>'

def inlineCSS(html):
    return instance.transform(html.replace('</head>', style + '</head>'))

def crawl(tabEL):
    payload = { 'curid': str(tabEL['pgId']) }
    html = requests.get(BASE_URL, params=payload).text
    htmlWithInlineCSS = inlineCSS(str(html))
    return htmlWithInlineCSS

In [ ]:
pageIDSample['HTML'] = pageIDSample.apply(crawl, axis='columns')

In [206]:
pageIDSample['HTML'] = pageIDSample['HTML'].str.replace('\n', '')
pageIDSample['HTML'] = pageIDSample['HTML'].str.replace('\t', '')

Since crawling is time expensive we store the data into file and db first.

In [17]:
client = MongoClient()
db = client.bob
pages = db.pages
pages.insert_many(pageIDSample.to_dict('records'))
pageIDSample.to_json(os.path.join("data", "crawled.json"))
client.close()

In [2]:
client = MongoClient()
db = client.bob
pages = db.pages
cursor = pages.find({})
pageIDSample = pd.DataFrame(list(cursor))
client.close()

In [18]:
pageIDSample.head()

,HTML,pgId
0,"<!DOCTYPE html><html class=""client-nojs"" lang=...",10041828
1,"<!DOCTYPE html><html class=""client-nojs"" lang=...",10086127
2,"<!DOCTYPE html><html class=""client-nojs"" lang=...",1008145
3,"<!DOCTYPE html><html class=""client-nojs"" lang=...",1012548
4,"<!DOCTYPE html><html class=""client-nojs"" lang=...",10128185


Now we extract the tables along with some metadata. For each row we assign an unique ID (the index of the row within the table) and a tag (whether the row includes `th-tags` only or is contained within a `thead`)

In [ ]:
HEADLINE_PATTERN = re.compile('(h|H)\d')
LABEL_CONTROLS = [
    {
        'label': 'Header',
        'color': 'light-blue'
    }, {
        'label': 'Data',
        'color': 'lime'
    }, {
        'label': 'Other',
        'color': 'orange'
    }
];

def extractPageTitle(soup):
    headlines = soup.select('h1')
    return headlines[0].text if len(headlines) > 0 else 'N/A'

def extractTableTitle(table):
    for sibling in table.previous_siblings:
        if (sibling is not None and sibling.name is not None and HEADLINE_PATTERN.match(sibling.name)):
            return sibling.text
    return 'N/A'

def addLabelControls(row, rowIndex, soup):
    labelControlTag = soup.new_tag(
        'th',
        attrs={
            'class': 'flex space-evenly'
        }
    )
    for labelControl in LABEL_CONTROLS:
        labelControlButton = soup.new_tag(
            'a',
            attrs={
                'class': 'labelButton waves-effect waves-light btn-small ' + labelControl['color'],
                'onClick': 'annotate(' + str(rowIndex) + ', "' + labelControl['label'] + '");',
            }
        )
        labelControlButton.string = labelControl['label']
        labelControlTag.append(labelControlButton)
    row.insert(0, labelControlTag)
    
def tagRow(row, rowIndex, soup, isHead=False):
    row['data-label'] = 'Header' if isHead else 'Data'
    row['data-row-index'] = rowIndex
    addLabelControls(row, rowIndex, soup)
    
def isHeaderRow(row):
    thTags = row.find_all('th', recursive=False)
    childCount = len(row.contents)
    return childCount == len(thTags) or row.parent.name == 'thead'

def tagRows(table, soup):
    rows = table.find_all('tr')
    annotations = []
    for rowIndex, row in enumerate(rows):
        isHeader = isHeaderRow(row)
        tagRow(row, rowIndex, soup, isHeader)  
        annotations.append('Header' if isHeader else 'Data')
    return annotations

def removeTableWidthLimitation(table):
    if not table.has_attr('style'):
        return
    tableStyle = parseStyle(table['style'])
    tableStyle['width'] = '100%'
    tableStyle['font-size'] = '100%'
    table['style'] = tableStyle.cssText
        
def extractTableInformation(table, pageID, tableIndex, pageTitle, soup):
    extractedInformation = {
        'pageID': pageID,
        'tableIndex': tableIndex,
        'pageTitle': pageTitle
    }
    extractedInformation['html'] = table.prettify()
    annotations = tagRows(table, soup)
    removeTableWidthLimitation(table)
    extractedInformation['taggedHtml'] = table.prettify()
    extractedInformation['annotations'] = annotations
    extractedInformation['tableTitle'] = extractTableTitle(table)
    return extractedInformation

def hasNestedTable(table):
    return len(table.select('table')) > 0

def extractTables(page):
    soup = BeautifulSoup(page['HTML'])
    pageTitle = extractPageTitle(soup)
    wikiTables = soup.select('.wikitable')
    extractedTables = []
    for tableIndex, table in enumerate(wikiTables):
        if hasNestedTable(table):
            continue
        extractedTable = extractTableInformation(table, page['pgId'], tableIndex, pageTitle, soup)
        extractedTables.append(extractedTable)
    return extractedTables

In [20]:
client = MongoClient()
db = client.bob
tables = db.tables
for extractedTables in pageIDSample.apply(extractTables, axis='columns').values:
    if len(extractedTables) > 0:
        tables.insert_many(extractedTables)
client.close()

ERROR	Property: Invalid value for "CSS Level 2.1" property: 1000 [1:1: width]
WARNING	Property: Unknown Property name. [1:1: cell-padding]


The data can now get labeled using the provided [labeling tool](https://github.com/RichStone/web-tables-header-detection/tree/master/Labeling%20Tool).

# Feature Extraction

In [23]:
client = MongoClient()
db = client.bob
tables = db.tables
cursor = tables.find({})
tables = pd.DataFrame(list(cursor))
client.close()

In [24]:
tables.head()

,_id,annotations,html,pageID,pageTitle,tableIndex,tableTitle,taggedHtml
0,5cf836ef1ae12a2caf23946f,"[Header, Header, Header, Data]","<table class=""wikitable floatright"" style=""flo...",10041828,Memories & Dust,0,N/A,"<table class=""wikitable floatright"" style=""flo..."
1,5cf836ef1ae12a2caf239470,"[Header, Data]","<table class=""wikitable"">\n <tbody>\n <tr>\n ...",10041828,Memories & Dust,1,Charts[edit],"<table class=""wikitable"">\n <tbody>\n <tr dat..."
2,5cf836ef1ae12a2caf239471,"[Header, Data]","<table class=""wikitable"">\n <tbody>\n <tr>\n ...",10086127,Sant Esteve de Palautordera,0,Demography[edit],"<table class=""wikitable"">\n <tbody>\n <tr dat..."
3,5cf836ef1ae12a2caf239472,"[Header, Data, Data, Data, Data]","<table class=""wikitable"" style=""text-align:cen...",1008145,Slovenia national football team,0,Euro 2000[edit],"<table class=""wikitable"" style=""text-align: ce..."
4,5cf836ef1ae12a2caf239473,"[Header, Data, Data, Data, Data]","<table class=""wikitable"" style=""text-align:cen...",1008145,Slovenia national football team,1,2002 World Cup[edit],"<table class=""wikitable"" style=""text-align: ce..."


In [29]:
SHORT_TEXT_THRESHOLD = 20
LONG_TEXT_THRESHOLD = 40

def isInt(value):
    try: 
        int(value)
        return True
    except ValueError:
        return False
    
def getRowSpan(cell):
    if cell.has_attr('rowspan') and isInt(cell['rowspan']):
        return int(cell['rowspan'])
    return 1
    
def getColSpan(cell):
    if cell.has_attr('colspan') and isInt(cell['colspan']):
        return int(cell['colspan'])
    return 1

def isMerged(cell):
    return (
        getColSpan(cell) > 1 or
        getRowSpan(cell) > 1
    )

def isCenterAligned(cell, style):
    return (
        (cell.has_attr('align') and cell['align'] == 'center') or
        (style is not None and 'text-align' in style and style['text-align'] == 'center')
    )

def isThOrInTHead(cell):
    row = cell.parent
    rowParent = row.parent
    return (
        cell.name == 'th' or
        rowParent.name == 'thead'
    )

def extractLayoutFeatures(cell, style):
    return {
        'isMerged': isMerged(cell),
        'isCenterAligned': isCenterAligned(cell, style),
        'isTHOrInTHead': isThOrInTHead(cell)
    }

def isBold(cell, style):
    return bool(
        style is not None and (
            style['font-weight'] == 'bold' or 
            style['font-style'] == 'bold'
        ) or
        cell.find('b') or
        cell.find('strong')
    )

def isItalic(cell, style):
    return bool(cell.find('i'))

def isUnderlined(cell, style):
    return (
        cell.find('u') or
        style is not None and (
            style['text-decoration'] == 'underline' or
            style['font-style'] == 'bold'
        )
    )

def isColored(cell, style):
    return (
        style is not None and (
            'background-color' in style or
            'color' in style
        )
    )

def extractStyleFeatures(cell, style):
    return {
        'isBold': isBold(cell, style),
        'isItalic': isItalic(cell, style),
        'isUnderlined': isUnderlined(cell, style)
    }

def getCellStyle(cell):
    return parseStyle(cell['style']) if cell.has_attr('style') else None

def getContentLength(cell):
    return len(re.sub('\s+',' ', cell.get_text()).split())

def isEmpty(cell):
    return getContentLength(cell) == 0

def isText(cell):
    return cell.get_text().isalpha()

def isNumeric(cell):
    return cell.get_text().isdigit()

def isDate(cell):
    try: 
        parse(cell.get_text(), fuzzy=False)
        return True
    except (ValueError, OverflowError):
        return False
    
def isShortText(cell):
    return getContentLength(cell) <= SHORT_TEXT_THRESHOLD

def isLongText(cell):
    return getContentLength(cell) > LONG_TEXT_THRESHOLD

def isTotal(cell):
    return cell.get_text().lower() == 'total'

def extractValueFeatures(cell):
    return {
        'isEmpty': isEmpty(cell),
        'isText': isText(cell),
        'isNumeric': isNumeric(cell),
        'isDate': isDate(cell),
        'isShortText': isShortText(cell),
        'isLongText': isLongText(cell),
        'isTotal': isTotal(cell)
    }

def mapDictBoolValuesToInt(dictionary):
    return { key: int(value) for key, value in dictionary.items() }

def applyColSpanFactor(dictionary, colSpan):
    return { key: value * colSpan for key, value in dictionary.items() }

def extractCellFeatures(cell, startRowIndex):
    cellStyle = getCellStyle(cell)
    boolCellFeatures = {
        **extractLayoutFeatures(cell, cellStyle),
        **extractStyleFeatures(cell, cellStyle),
        **extractValueFeatures(cell)
    }
    intCellFeatures = mapDictBoolValuesToInt(boolCellFeatures)
    # intCellFeatures = applyColSpanFactor(intCellFeatures, getColSpan(cell))
    cellFeatures = []
    rowSpan = getRowSpan(cell)
    for rowIndex in range(startRowIndex, startRowIndex + rowSpan):
        featureCopy = dict(intCellFeatures)
        featureCopy['row'] = rowIndex
        featureCopy['colCount'] = 1
        cellFeatures.append(featureCopy)
    return cellFeatures

def merge(featuresA, featuresB):
    return { k: featuresA.get(k, 0) + featuresB.get(k, 0) for k in set(featuresA) | set(featuresB) }

def stringifyDictKeys(dictionary):
    return { str(key): value for key, value in dictionary.items() }

def numNormalisedCols(row):
    numCols = 0
    for cell in row.children:
        if type(cell) is Tag:
            numCols += getColSpan(cell)
    return numCols
            
def getSimilarity(feature, cell, neighbour, suffix):
    similarity = {}
    similarity[feature + 'A' + suffix] = cell[feature] and neighbour[feature]
    similarity[feature + 'B' + suffix] = cell[feature] and not neighbour[feature]
    return similarity
    
def extractSimilarityFeatures(cell, neighbour, suffix):
    similarityFeatures = {
        **getSimilarity('isMerged', cell, neighbour, suffix),
        **getSimilarity('isCenterAligned', cell, neighbour, suffix),
        **getSimilarity('isTHOrInTHead', cell, neighbour, suffix),
        **getSimilarity('isBold', cell, neighbour, suffix),
        **getSimilarity('isItalic', cell, neighbour, suffix),
        **getSimilarity('isUnderlined', cell, neighbour, suffix),
        **getSimilarity('isEmpty', cell, neighbour, suffix),
        **getSimilarity('isText', cell, neighbour, suffix),
        **getSimilarity('isNumeric', cell, neighbour, suffix),
        **getSimilarity('isDate', cell, neighbour, suffix),
        **getSimilarity('isShortText', cell, neighbour, suffix),
        **getSimilarity('isLongText', cell, neighbour, suffix),
        **getSimilarity('isTotal', cell, neighbour, suffix)        
    }
    return similarityFeatures    
    
def addSimilarityFeatures(normalizedFeatureTable):
    nftWithSimilarity = []
    numRows = len(normalizedFeatureTable)
    for rowIndex, row in enumerate(normalizedFeatureTable):
        nftWithSimilarity.append([])
        for cellIndex, cell in enumerate(row):
            features = cell
            if rowIndex > 0:
                features = {
                    **extractSimilarityFeatures(cell, normalizedFeatureTable[rowIndex - 1][cellIndex], 'u'), 
                    **features
                    }
            if rowIndex < numRows - 1:
                features = {
                    **extractSimilarityFeatures(cell, normalizedFeatureTable[rowIndex + 1][cellIndex], 'l'),
                    **features
                }
            intCellFeatures = mapDictBoolValuesToInt(features)
            nftWithSimilarity[-1].append(intCellFeatures)
    return nftWithSimilarity
    
def normalizedFeatureTable(table):
    soup = BeautifulSoup(table['html'])
    rows = soup.select('tr')
    numRows = len(rows)
    numCols = numNormalisedCols(rows[0])
    nft = [['empty cell' for i in range(numCols)] for j in range(numRows)]
    for rowIndex, row in enumerate(rows):
        cellIndex = 0
        for cell in row.children:
            if type(cell) is not Tag:
                continue
            cellStyle = getCellStyle(cell)
            boolCellFeatures = {
                **extractLayoutFeatures(cell, cellStyle),
                **extractStyleFeatures(cell, cellStyle),
                **extractValueFeatures(cell)
            }
            colSpan = getColSpan(cell)
            rowSpan = getRowSpan(cell)
            # find next empty cell
            while cellIndex < numCols and nft[rowIndex][cellIndex] != 'empty cell':
                cellIndex += 1
            for rIndex in range(rowIndex, rowIndex + rowSpan):
                for cIndex in range(cellIndex, cellIndex + colSpan):
                    nft[rIndex][cIndex] = boolCellFeatures
            cellIndex += colSpan
    return addSimilarityFeatures(nft)    
    
def extractFeaturesNormalized(table):
    featureTable = normalizedFeatureTable(table)
    rowFeatureTable = {}
    for rowIndex, row in enumerate(featureTable):
        # count how often every feature is true in a row
        rowFeatures = {}
        for cellFeatures in row:
            rowFeatures = merge(rowFeatures, cellFeatures)
        rowFeatureTable[rowIndex] = rowFeatures
    rowFeatureTable = stringifyDictKeys(rowFeatureTable)
    return rowFeatureTable
    
def extractFeatures(table):
    soup = BeautifulSoup(table['html'])
    rows = soup.select('tr')
    rowsFeatures = {}
    for rowIndex, row in enumerate(rows):
        for cell in row.children:
            if type(cell) is not Tag:
                continue
            for rowFeatures in extractCellFeatures(cell, rowIndex):
                rowFeatureIndex = rowFeatures.pop('row')
                merged = merge(rowsFeatures.get(rowFeatureIndex, {}), rowFeatures)
                rowsFeatures[rowFeatureIndex] = merged
    rowsFeatures = stringifyDictKeys(rowsFeatures)
    return rowsFeatures

In [33]:
testHtmlTable = {}
testHtmlTable['html'] = """<table>
  <tr>
    <th>Month</th>
    <th>Savings</th>
    <th>Savings for holiday!</th>
    <th>Test</th>
  </tr>
  <tr>
    <td>January</td>
    <td>$100</td>
    <td rowspan="2">$50</td>
    <td> test</td>
  </tr>
  <tr>
    <td>February</td>
    <td>$80</td>
    <td>test</td>
  </tr>
</table>"""

featureTable = extractFeaturesNormalized(testHtmlTable)
for rowIndex in featureTable:
    print(rowIndex)
    print(featureTable[rowIndex])

0
{'isMergedBl': 0, 'isEmpty': 0, 'isTHOrInTHeadAl': 0, 'isNumeric': 0, 'isEmptyBl': 0, 'isCenterAligned': 0, 'isBoldBl': 0, 'isUnderlinedBl': 0, 'isNumericBl': 0, 'isDateAl': 0, 'isLongTextBl': 0, 'isLongText': 0, 'isItalicBl': 0, 'isShortTextAl': 4, 'isTHOrInTHeadBl': 4, 'isMergedAl': 0, 'isTotalAl': 0, 'isUnderlinedAl': 0, 'isBoldAl': 0, 'isTotalBl': 0, 'isBold': 0, 'isCenterAlignedAl': 0, 'isItalicAl': 0, 'isItalic': 0, 'isEmptyAl': 0, 'isCenterAlignedBl': 0, 'isShortText': 4, 'isShortTextBl': 0, 'isUnderlined': 0, 'isNumericAl': 0, 'isTotal': 0, 'isTHOrInTHead': 4, 'isTextBl': 2, 'isTextAl': 1, 'isDateBl': 0, 'isMerged': 0, 'isText': 3, 'isDate': 0, 'isLongTextAl': 0}
1
{'isUnderlinedBu': 0, 'isMergedBl': 0, 'isTotalAu': 0, 'isEmpty': 0, 'isTHOrInTHeadAl': 0, 'isShortTextAu': 4, 'isCenterAligned': 0, 'isBoldBl': 0, 'isDateBu': 1, 'isNumericBl': 0, 'isLongTextAu': 0, 'isLongText': 0, 'isShortTextBu': 0, 'isTotalBu': 0, 'isShortTextAl': 4, 'isBoldAu': 0, 'isTHOrInTHeadBl': 0, 'isMer

In [26]:
tables['features'] = tables.apply(extractFeatures, axis='columns')
tables.head()

ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERRO

ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: linear-gradient(transparent, transparent), url("data:image/svg+xml,%3Csvg xmlns=%22http://www.w3.org/2000/svg%22 width=%2221%22 height=%229%22 viewBox=%220 0 21 9%22%3E %3Cpath d=%22M14.5 5l-4 4-4-4zm0-1l-4-4-4 4z%22/%3E %3C/svg%3E") [1:1: background-image]
ERROR	Property: In

ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERRO

ERROR	Property: Invalid value for "CSS Level 2.1" property: linear-gradient(transparent, transparent), url("data:image/svg+xml,%3Csvg xmlns=%22http://www.w3.org/2000/svg%22 width=%2221%22 height=%229%22 viewBox=%220 0 21 9%22%3E %3Cpath d=%22M14.5 5l-4 4-4-4zm0-1l-4-4-4 4z%22/%3E %3C/svg%3E") [1:1: background-image]
ERROR	Property: Invalid value for "CSS Level 2.1" property: linear-gradient(transparent, transparent), url("data:image/svg+xml,%3Csvg xmlns=%22http://www.w3.org/2000/svg%22 width=%2221%22 height=%229%22 viewBox=%220 0 21 9%22%3E %3Cpath d=%22M14.5 5l-4 4-4-4zm0-1l-4-4-4 4z%22/%3E %3C/svg%3E") [1:1: background-image]
ERROR	Property: Invalid value for "CSS Level 2.1" property: linear-gradient(transparent, transparent), url("data:image/svg+xml,%3Csvg xmlns=%22http://www.w3.org/2000/svg%22 width=%2221%22 height=%229%22 viewBox=%220 0 21 9%22%3E %3Cpath d=%22M14.5 5l-4 4-4-4zm0-1l-4-4-4 4z%22/%3E %3C/svg%3E") [1:1: background-image]
ERROR	Property: Invalid value for "CSS Level 2

ERROR	Property: Invalid value for "CSS Level 2.1" property: linear-gradient(transparent, transparent), url("data:image/svg+xml,%3Csvg xmlns=%22http://www.w3.org/2000/svg%22 width=%2221%22 height=%229%22 viewBox=%220 0 21 9%22%3E %3Cpath d=%22M14.5 5l-4 4-4-4zm0-1l-4-4-4 4z%22/%3E %3C/svg%3E") [1:1: background-image]
ERROR	Property: Invalid value for "CSS Level 2.1" property: linear-gradient(transparent, transparent), url("data:image/svg+xml,%3Csvg xmlns=%22http://www.w3.org/2000/svg%22 width=%2221%22 height=%229%22 viewBox=%220 0 21 9%22%3E %3Cpath d=%22M14.5 5l-4 4-4-4zm0-1l-4-4-4 4z%22/%3E %3C/svg%3E") [1:1: background-image]
ERROR	Property: Invalid value for "CSS Level 2.1" property: linear-gradient(transparent, transparent), url("data:image/svg+xml,%3Csvg xmlns=%22http://www.w3.org/2000/svg%22 width=%2221%22 height=%229%22 viewBox=%220 0 21 9%22%3E %3Cpath d=%22M14.5 5l-4 4-4-4zm0-1l-4-4-4 4z%22/%3E %3C/svg%3E") [1:1: background-image]
ERROR	No content to parse.
ERROR	PropertyValue

...

ERROR	Property: No property value found: background-color: [1:17: :]
ERROR	CSSStyleDeclaration: Syntax Error in Property: background-color:
ERROR	Property: No property value found: background-color: [1:17: :]
ERROR	CSSStyleDeclaration: Syntax Error in Property: background-color:
WARNING	Property: Unknown Property name. [1:34: -moz-box-shadow]
WARNING	Property: Unknown Property name. [1:103: -webkit-box-shadow]
WARNING	Property: Unknown Property name. [1:34: -moz-box-shadow]
WARNING	Property: Unknown Property name. [1:103: -webkit-box-shadow]
WARNING	Property: Unknown Property name. [1:34: -moz-box-shadow]
WARNING	Property: Unknown Property name. [1:103: -webkit-box-shadow]
WARNING	Property: Unknown Property name. [1:34: -moz-box-shadow]
WARNING	Property: Unknown Property name. [1:103: -webkit-box-shadow]
WARNING	Property: Unknown Property name. [1:34: -moz-box-shadow]
WARNING	Property: Unknown Property name. [1:103: -webkit-box-shadow]
WARNING	Property: Unknown Property name. [1:34: -m

INFO	CSSStyleDeclaration: Stripped standalone semicolon: ;
INFO	CSSStyleDeclaration: Stripped standalone semicolon: ;
INFO	CSSStyleDeclaration: Stripped standalone semicolon: ;
ERROR	Property: Invalid value for "CSS Level 2.1" property: center [1:57: vertical-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: center [1:57: vertical-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: center [1:57: vertical-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: center [1:57: vertical-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: center [1:57: vertical-align]
WARNING	Property: Unknown Property name. [1:20: ext-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: center [1:56: vertical-align]
WARNING	Property: Unknown Property name. [1:20: text-aligneight]
ERROR	Property: Invalid value for "CSS Level 2.1" property: center [1:43: vertical-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: center [1

ERROR	PropertyValue: No match: ('CHAR', ':', 1, 39)
ERROR	PropertyValue: Unknown syntax or no value: border-top:1px solid darkgray
ERROR	CSSStyleDeclaration: Syntax Error in Property: border-top:border-top:1px solid darkgray
ERROR	PropertyValue: No match: ('CHAR', ':', 1, 39)
ERROR	PropertyValue: Unknown syntax or no value: border-top:1px solid darkgray
ERROR	CSSStyleDeclaration: Syntax Error in Property: border-top:border-top:1px solid darkgray
ERROR	PropertyValue: No match: ('CHAR', ':', 1, 54)
ERROR	PropertyValue: Unknown syntax or no value: border-top:1px solid darkgray
ERROR	CSSStyleDeclaration: Syntax Error in Property: border-top:border-top:1px solid darkgray
ERROR	PropertyValue: No match: ('CHAR', ':', 1, 22)
ERROR	PropertyValue: Unknown syntax or no value: border-top:1px solid darkgray
ERROR	CSSStyleDeclaration: Syntax Error in Property: border-top:border-top:1px solid darkgray
ERROR	PropertyValue: No match: ('CHAR', ':', 1, 39)
ERROR	PropertyValue: Unknown syntax or no value:

ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERROR	Property: Invalid value for "CSS Level 2.1" property: auto [1:1: text-align]
ERRO

,_id,annotations,html,pageID,pageTitle,tableIndex,tableTitle,taggedHtml,features
0,5cf836ef1ae12a2caf23946f,"[Header, Header, Header, Data]","<table class=""wikitable floatright"" style=""flo...",10041828,Memories & Dust,0,N/A,"<table class=""wikitable floatright"" style=""flo...","{'0': {'isEmpty': 0, 'isBold': 0, 'colCount': ..."
1,5cf836ef1ae12a2caf239470,"[Header, Data]","<table class=""wikitable"">\n <tbody>\n <tr>\n ...",10041828,Memories & Dust,1,Charts[edit],"<table class=""wikitable"">\n <tbody>\n <tr dat...","{'0': {'isBold': 0, 'colCount': 3, 'isTHOrInTH..."
2,5cf836ef1ae12a2caf239471,"[Header, Data]","<table class=""wikitable"">\n <tbody>\n <tr>\n ...",10086127,Sant Esteve de Palautordera,0,Demography[edit],"<table class=""wikitable"">\n <tbody>\n <tr dat...","{'0': {'isBold': 0, 'colCount': 6, 'isTHOrInTH..."
3,5cf836ef1ae12a2caf239472,"[Header, Data, Data, Data, Data]","<table class=""wikitable"" style=""text-align:cen...",1008145,Slovenia national football team,0,Euro 2000[edit],"<table class=""wikitable"" style=""text-align: ce...","{'0': {'isBold': 0, 'colCount': 11, 'isTHOrInT..."
4,5cf836ef1ae12a2caf239473,"[Header, Data, Data, Data, Data]","<table class=""wikitable"" style=""text-align:cen...",1008145,Slovenia national football team,1,2002 World Cup[edit],"<table class=""wikitable"" style=""text-align: ce...","{'0': {'isBold': 0, 'colCount': 9, 'isTHOrInTH..."


In [28]:
client = MongoClient()
db = client.bob
tablesCollection = db.tables
tablesCollection.insert_many(tables.to_dict('records'))
client.close()

## Logarithmic Binning

In [5]:
client = MongoClient()
db = client.bob
tables = db.tables
cursor = tables.find({})
tables = pd.DataFrame(list(cursor))
client.close()
tables.head()

,_id,annotatedAt,annotations,features,html,logBin,pageID,pageTitle,tableIndex,tableTitle,taggedHtml
0,5cf28fdb1ae12a2691e6c562,1.559651e+12,"[Header, Header, Header, Data]","{'0': {'isBold': 0, 'isItalic': 0, 'isShortTex...","<table class=""wikitable floatright"" style=""flo...","[{'isBold': True, 'isItalic': True, 'isShortTe...",10041828,Memories & Dust,0,N/A,"<table class=""wikitable floatright"" style=""flo..."
1,5cf28fdb1ae12a2691e6c563,1.559651e+12,"[Header, Data]","{'0': {'isItalic': 0, 'isShortText': 3, 'isCen...","<table class=""wikitable"">\n <tbody>\n <tr>\n ...","[{'isItalic': True, 'isShortText': True, 'isCe...",10041828,Memories & Dust,1,Charts[edit],"<table class=""wikitable"">\n <tbody>\n <tr dat..."
2,5cf28fdb1ae12a2691e6c564,1.559651e+12,"[Header, Data]","{'0': {'isItalic': 0, 'isShortText': 6, 'isCen...","<table class=""wikitable"">\n <tbody>\n <tr>\n ...","[{'isItalic': True, 'isShortText': True, 'isCe...",10086127,Sant Esteve de Palautordera,0,Demography[edit],"<table class=""wikitable"">\n <tbody>\n <tr dat..."
3,5cf28fdb1ae12a2691e6c565,1.559651e+12,"[Header, Data, Data, Data, Data]","{'0': {'isItalic': 0, 'isShortText': 11, 'isCe...","<table class=""wikitable"" style=""text-align:cen...","[{'isItalic': True, 'isShortText': True, 'isCe...",1008145,Slovenia national football team,0,Euro 2000[edit],"<table class=""wikitable"" style=""text-align: ce..."
4,5cf28fdb1ae12a2691e6c566,1.559651e+12,"[Header, Data, Data, Data, Data]","{'0': {'isItalic': 0, 'isShortText': 9, 'isCen...","<table class=""wikitable"" style=""text-align:cen...","[{'isItalic': True, 'isShortText': True, 'isCe...",1008145,Slovenia national football team,1,2002 World Cup[edit],"<table class=""wikitable"" style=""text-align: ce..."


In [27]:
def calcA(c, r):
    if (c == 0):
        return 0
    if (c == r):
        return r
    if (c > r / 2.0):
        return math.floor(math.log2(r - c) + 1)
    return math.floor(math.log2(c) + 1)

def calcB(c, r):
    return math.floor(math.log2(r))

def isInSameBin(rowA, rowB, featureKey):
    return (
        calcB(rowA[featureKey], rowA['colCount']) == calcB(rowB[featureKey], rowB['colCount']) and 
        calcA(rowA[featureKey], rowA['colCount']) == calcA(rowB[featureKey], rowB['colCount'])
    )

def logBinTable(table):
    logBins = {}
    for rowIndex, row in table['features'].items():
        logBin = dict(row)
        colCount = logBin.pop('colCount')
        logBin = { 
            featureKey: { 
                'a': calcA(feature, colCount),
                'b': calcB(feature, colCount)
            } for featureKey, feature in logBin.items() 
        }
        logBins[rowIndex] = logBin
    return logBins

In [28]:
tables['logBin'] = tables.apply(logBinTable, axis='columns')
tables.head()

,_id,annotations,html,pageID,pageTitle,tableIndex,tableTitle,taggedHtml,features,logBin
0,5cf836ef1ae12a2caf23946f,"[Header, Header, Header, Data]","<table class=""wikitable floatright"" style=""flo...",10041828,Memories & Dust,0,N/A,"<table class=""wikitable floatright"" style=""flo...","{'0': {'isEmpty': 0, 'isBold': 0, 'colCount': ...","{'0': {'isEmpty': {'a': 0, 'b': 0}, 'isBold': ..."
1,5cf836ef1ae12a2caf239470,"[Header, Data]","<table class=""wikitable"">\n <tbody>\n <tr>\n ...",10041828,Memories & Dust,1,Charts[edit],"<table class=""wikitable"">\n <tbody>\n <tr dat...","{'0': {'isBold': 0, 'colCount': 3, 'isTHOrInTH...","{'0': {'isBold': {'a': 0, 'b': 1}, 'isTHOrInTH..."
2,5cf836ef1ae12a2caf239471,"[Header, Data]","<table class=""wikitable"">\n <tbody>\n <tr>\n ...",10086127,Sant Esteve de Palautordera,0,Demography[edit],"<table class=""wikitable"">\n <tbody>\n <tr dat...","{'0': {'isBold': 0, 'colCount': 6, 'isTHOrInTH...","{'0': {'isBold': {'a': 0, 'b': 2}, 'isTHOrInTH..."
3,5cf836ef1ae12a2caf239472,"[Header, Data, Data, Data, Data]","<table class=""wikitable"" style=""text-align:cen...",1008145,Slovenia national football team,0,Euro 2000[edit],"<table class=""wikitable"" style=""text-align: ce...","{'0': {'isBold': 0, 'colCount': 11, 'isTHOrInT...","{'0': {'isBold': {'a': 0, 'b': 3}, 'isTHOrInTH..."
4,5cf836ef1ae12a2caf239473,"[Header, Data, Data, Data, Data]","<table class=""wikitable"" style=""text-align:cen...",1008145,Slovenia national football team,1,2002 World Cup[edit],"<table class=""wikitable"" style=""text-align: ce...","{'0': {'isBold': 0, 'colCount': 9, 'isTHOrInTH...","{'0': {'isBold': {'a': 0, 'b': 3}, 'isTHOrInTH..."


In [29]:
client = MongoClient()
db = client.bob
tablesCollection = db.tables
tablesCollection.insert_many(tables.to_dict('records'))
client.close()

In [1]:
print("test")

test
